In [2]:
import nltk
import numpy
import tflearn
import tensorflow
import pickle
import random
import json
nltk.download('punkt')
   
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
  
 #loading the json data
with open("intents.json") as file:                  
    data = json.load(file)
      
#print(data["intents"])
try:
    with open("data.pickle", "rb") as f:
        words, l, training, output = pickle.load(f)
except:
      
    #  Extracting Data
    words = []
    l = []
    docs_x = []
    docs_y = []
      
   # converting each pattern into list of words using nltk.word_tokenizer 
    for i in data["intents"]:   
        for p in i["patterns"]:
            wrds = nltk.word_tokenize(p)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(i["tag"])
   
            if i["tag"] not in l:
                l.append(i["tag"])
    # Word Stemming            
    words = [stemmer.stem(w.lower()) for w in words if w != "?"]         
    words = sorted(list(set(words)))
    l = sorted(l)                                      
      
    # This code will simply create a unique list of stemmed 
    # words to use in the next step of our data preprocessing
    training = []
    output = []
    out_empty = [0 for _ in range(len(l))]
    for x, doc in enumerate(docs_x):
        bag = []
   
        wrds = [stemmer.stem(w) for w in doc]
   
        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)
        output_row = out_empty[:]
        output_row[l.index(docs_y[x])] = 1
   
        training.append(bag)
        output.append(output_row)
          
    # Finally we will convert our training data and output to numpy arrays    
    training = numpy.array(training)        
    output = numpy.array(output)
    with open("data.pickle", "wb") as f:
        pickle.dump((words, l, training, output), f)
  
          
# Developing a Model        
tensorflow.compat.v1.reset_default_graph()                    
   
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)
  
  
# remove comment to not train model after you satisfied with the accuracy
model = tflearn.DNN(net)
"""try:                               
    model.load("model.tflearn")
except:"""
  
# Training & Saving the Model
model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)        
model.save("model.tflearn")
   
# making predictions
def bag_of_words(s, words):                                
    bag = [0 for _ in range(len(words))]
   
    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]
   
    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
   
    return numpy.array(bag)

Training Step: 5999  | total loss: 0.25047 | time: 0.015s
| Adam | epoch: 1000 | loss: 0.25047 - acc: 0.9869 -- iter: 40/43
Training Step: 6000  | total loss: 0.22577 | time: 0.018s
| Adam | epoch: 1000 | loss: 0.22577 - acc: 0.9882 -- iter: 43/43
--
INFO:tensorflow:E:\machine learning\covid chatbot\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [5]:
def chat():
    print("""Start talking with the bot and ask your
    queries about Corona-virus(type quit to stop)!""")
      
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break
   
        results = model.predict([bag_of_words(inp, words)])[0]
        results_index = numpy.argmax(results)
          
        #print(results_index)
        tag = l[results_index]
        if results[results_index] > 0.7:
            for tg in data["intents"]:
                if tg['tag'] == tag:
                    responses = tg['responses']
   
            print(random.choice(responses))
        else:
            print("I am sorry but I can't understand")
   
chat()

Start talking with the bot and ask your
    queries about Corona-virus(type quit to stop)!
You: what is covid


UnboundLocalError: local variable 'responses' referenced before assignment

In [10]:
# !pip install beautifulsoup4
# !pip install google
try:
    from googlesearch import search
except ImportError:
    print("No module named 'google' found")
 
# to search
query = "Geeksforgeeks"
search_results=[]
for j in search(query, tld="co.in", num=10, stop=10, pause=2):
    search_results.append(j)
print(search_results[0])

https://www.geeksforgeeks.org/


In [25]:
from nltk.probability import FreqDist
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import nltk.data
from itertools import islice
from functools import cmp_to_key

class SimpleSummarizer:

	def reorder_sentences( self, output_sentences, input ):
		output_sentences.sort( key=cmp_to_key(lambda s1, s2:
			input.find(s1) - input.find(s2)) )
		return output_sentences
	
	def get_summarized(self, input, num_sentences ):
		# TODO: allow the caller to specify the tokenizer they want
		# TODO: allow the user to specify the sentence tokenizer they want
		
		tokenizer = RegexpTokenizer('\w+')
		
		# get the frequency of each word in the input
		base_words = [word.lower()
			for word in tokenizer.tokenize(input)]
		words = [word for word in base_words if word not in stopwords.words()]
		word_frequencies = FreqDist(words)
		
		# now create a set of the most frequent words
		most_frequent_words = [pair[0] for pair in
			islice(word_frequencies.items(), 0, 100)]
		
		# break the input up into sentences.  working_sentences is used
		# for the analysis, but actual_sentences is used in the results
		# so capitalization will be correct.
		
		sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
		actual_sentences = sent_detector.tokenize(input)
		working_sentences = [sentence.lower()
			for sentence in actual_sentences]

		# iterate over the most frequent words, and add the first sentence
		# that inclues each word to the result.
		output_sentences = []

		for word in most_frequent_words:
			for i in range(0, len(working_sentences)):
				if (word in working_sentences[i]
				  and actual_sentences[i] not in output_sentences):
					output_sentences.append(actual_sentences[i])
					break
				if len(output_sentences) >= num_sentences: break
			if len(output_sentences) >= num_sentences: break
			
		# sort the output sentences back to their original order
		return self.reorder_sentences(output_sentences, input)
	
	def summarize(self, input, num_sentences):
		return " ".join(self.get_summarized(input, num_sentences))

In [26]:
input = "NLTK is a python library for working human-written text. Summarize is a package that uses NLTK to create summaries."
x=SimpleSummarizer()
x.summarize(input,1)

'NLTK is a python library for working human-written text.'

In [82]:
test_data=["Are Africans immune against covid-19", "Are children under a certain age immune to the virus", "are coronavirus victims getting younger", "Are young people immune to COVID-19?", "Can a mosquito spread the Corona virus with its bite?", "Can exposure to sunshine kill the virus?", "Can herbal medicine help to slow spread of Covid19?", "Can i breathe in extra hot air to kill the virus?", "Can i get a tuberculosis shot to be immune to the virus?", "Can my dog get coronavirus?", "Can my pet get corona virus", "Can my pet get coronavirus?", "Can the transmission be done by mosquitos?", "Can you get the same corona virus more than one time", "Can you tell me any myths about the Corona Virus?", "Can young people get COVID-19?", "Corona and hot lime water.", "Covid 19 separating myths from facts.", "Covid19 and frequency of testing if there is common cold and flu.", "Covid19 and its allergic effects post 2 months of self quarantine.", "Covid19 and its facts.", "Covid19 and its self help via self quarantine immune booster.", "Covid19 and its treatments with alcohol.", "Covid19 and the doomsday.", "Covid19 and the misinformation industry.", "Covid19 cough syrups and inhalers.", "Covid19 hoaxes.", "Covid19 home remedies and hot fluids.", "Covid19 myths and lies.", "Did china find a cure for the virus and doesnt want to share with the world?", "did coronavirus begin in a lab", "did coronavirus come from a lab", "did coronavirus start in a lab", "Did Covid 19 come about as a result of 5G Technology?", "Do 5G mobile networks spread COVID-19?", "do black people get corona virus", "do chemtrails spread covid 19", "Do marijuana smokers get the virus worse than non marijuana smokers?", "Does drinking vitamin C prevent the corona", "Does drinking water flush out corona virus", "does limon help to cure corona virus", "Does race/ethnicity play a factor in the spread of the virus", "Does snake oil cure covid?", "Does the sun kill Corona virus?", "Does the virus spread more in cold countries or hot countries?", "Does the virus survive extreme heat?", "Has Cuba found a cure for the corona virus already?", "Has France found a cure for the corona virus already?", "Has Germany found a cure for the corona virus already?", "Has there been any research on mosquitoes linked to Covid19?", "How can I distinguish between rumor and facts about corona virus?", "How can I get trust-worthy information?", "How does eating dead animals give me the virus?", "I am wondering if eating garlic can help to prevent COVID-19.", "I heard if you gurgle with water and salt you will avoid catching the virus. Is this true?", "I want to know what is false about the virus", "I want to learn about the myths regarding the origin and spread of covid-19.", "I want to learn more about disinformation about covid-19", "I want to learn more about wide spread myths as regards covi.", "I would like to see research done for mosquitoes and the Corona virus.", "If there is no cure, how come people are being sent home", "Is China using covid-19 to take over the world?", "Is Corona a disease that was improvised in the labs?", "is corona virus a bio weapon", "is corona virus a lab leak", "Is corona virus caused by 5G?", "is coronavirus a hoax", "is coronavirus real", "Is covid-19 a biomedical weapon?", "Is Covid-19 a Chinese virus?", "Is COVID-19 Chinese?", "is covid-19 man made", "Is Covid-19 part of population control?", "is covid-19 real", "Is Covid-19 World War III?", "is it true corona virus does not effect caucasians", "Is it true that weather affects the spread of the virus so things will be alright when summer arrives?", "is the corona virus a communist propaganda", "Is the corona virus Chinese?", "is the corona virus immune to african american", "Is the government hiding information about spread of Corona?", "Is there a correlation between 5G and a weakened immune system?", "Look up \'5G\' and \'COVID-19\'", "Look up \'corona virus\' and \'conspiracy.\'", "Look up \'Zoom\' and \'COVID-19\'", "Real versus false facts about corona", "Search \'coronavirus hoax\' for me", "Search \'coronavirus myths\' for me", "Should I take hot baths?", "Show me examples of disinformation about covid-19", "Show me false information that is being spread about Covid-19.", "Show me myths about Covid19", "Tell me fake news about Corona", "Tell me more about the myths surrounding covid-19.", "was corona virus being weaponized", "was corona virus from a lab", "Was corona virus made in a lab?", "Was corona virus made in the lab", "was coronavirus a bioweapon", "was coronavirus bioengineered", "was coronavirus created", "was coronavirus created in lab", "was coronavirus intentional", "was coronavirus made in a lab", "was coronavirus man made", "was coronavirus manufactured", "was coronavirus released on purpose", "Was Covid19 created in a lab?", "was covid-19 made in a Chinese lab", "was covid-19 man made", "Was the virus caused by someone eating a bat?", "Was the virus created by Trump?", "What are the myths about corona", "What are the real effects of corona", "What home remedies should I try?", "what is china spending money on to help corona virus", "What is not true about the virus", "What is true and what is not true about the virus?", "What myths are there about the corona virus?", "What myths are there on covid-19?", "Where can I check covid-19 facts and myths?", "Where can I find out whether or not some hoaxes are true?", "WHO Covid19 hoaxes.", "Who is spreading the virus more, men or women?", "Why are people saying Covid-19 is a Chinese virus?", "why did the corona virus break out", "Will  Corona virus disappear in Spring?", "Will a Covid19 vaccine be dangerous to take?", "Will drinking alcohol protect me against COVID-19?", "Will ginger and lemon help in prevention?"]

In [94]:
try:
    from googlesearch import search
except ImportError:
    print("No module named 'google' found")
 
# to search
query = "Are fruits, vegetables and herbs useful in fighting the coronavirus?"
print(query)
search_results=[]
for j in search(query, tld="co.in", num=10, stop=10, pause=2):
    search_results.append(j)


import requests
from bs4 import BeautifulSoup

url = search_results[0]
print(url)
response = requests.get(url)
soup = BeautifulSoup(response.text)
print(soup)

# fetch_result=[meta.attrs['content'] for meta in metas if 'name' in meta.attrs and meta.attrs['name'] == 'description']
# print(fetch_result)
# x.summarize(fetch_result[0],1)

Are fruits, vegetables and herbs useful in fighting the coronavirus?
https://newseu.cgtn.com/news/2021-03-26/Eight-fruits-vegetables-and-herbs-that-might-help-tackle-COVID-19-YVuDwvMN5m/index.html
<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<title>Eight fruits, vegetables and herbs that might help tackle COVID-19 - CGTN</title>
<meta content="width=device-width,minimum-scale=1,initial-scale=1,user-scalable=no" name="viewport"/>
<meta content="" name="description"/>
<meta content="" name="author"/>
<meta content="" name="keywords"/>
<meta content="no-transform" http-equiv="Cache-Control"/>
<meta content="no-siteapp" http-equiv="Cache-Control"/>
<meta content="telephone=no" name="format-detection"/>
<meta content="yes" name="apple-mobile-web-app-capable"/>
<meta content="black" name="apple-mobile-web-app-status-bar-style"/>
<meta content="IE=Edge,chrome=1" http-equiv="X-UA-Compatible"/>
<!-- FB -->
<meta content="565225540184937" property="fb:pages"/>
<!--(twittercard

In [100]:
metas = soup.find_all('meta') #Get Meta Description
flag=0
for m in metas:
    x=m.get('name')
    if x and x == 'Description' or x=='description':
        desc = m.get('content')
        if desc=="":
            pass
        else:
            flag=1
            print(desc)
        break
if(flag==0):
    for m in metas:
        x=m.get('name')
        if x and (x.find('Description') or x.find('description'))>=0:
            desc = m.get('content')
            flag=1
            print(desc)
            break
if(flag==0):
    print("sorry i dont know")

sorry i dont know
